# Smart Agent Chatbot: Clarification + Brief Generation

This notebook builds a LangGraph chatbot that asks clarification questions and then generates a research brief.


In [1]:
%pip install langgraph langchain-google-genai python-dotenv


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
%pip install --upgrade langchain-google-genai


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
import os
from dotenv import load_dotenv
from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

api_key = os.getenv("GOOGLE_API_KEY") or os.getenv("GEMINI_API_KEY")
if not api_key:
    raise RuntimeError("Set GOOGLE_API_KEY or GEMINI_API_KEY in .env file.")

# Initialize model
llm = ChatGoogleGenerativeAI(
    model="models/gemini-2.5-flash", 
    google_api_key=api_key, 
    temperature=0.7
)
                                                                                                                                                                                                                                                  

In [16]:
#State Schema
class State(TypedDict):
    user_request: str | None
    topic: str | None
    scope: str | None
    time_period: str | None
    preferred_sources: str | None
    format: str | None
    brief: str | None
    messages: Annotated[list, add_messages]


In [17]:
#Nodes: Check for Missing Field, Ask, Receive, Generate
def check_missing(state: State):
    required = ["topic", "scope", "time_period", "preferred_sources", "format"]
    for field in required:
        if not state.get(field):
            return {"missing_field": field}
    return {"missing_field": None}

def ask_clarification(state: State, missing_field: str):
    questions = {
        "topic": "What topic should the research be about?",
        "scope": "Do you want detailed or overview scope?",
        "time_period": "What time period should be covered?",
        "preferred_sources": "Any preferred sources (news, academic, blogs)?",
        "format": "In what format should the brief be (bullet-points / essay / comparison)?"
    }
    return {"question": questions.get(missing_field, "Can you clarify more?")}

def receive_input(state: State, missing_field: str, answer: str):
    state[missing_field] = answer.strip()
    return {}

def generate_brief(state: State):
    prompt = f"""
You are to write a research brief based on:
- Topic: {state['topic']}
- Scope: {state['scope']}
- Time Period: {state['time_period']}
- Preferred Sources: {state['preferred_sources']}
- Format: {state['format']}
User Request: {state['user_request']}
"""
    # you can include messages history if needed
    resp = llm.invoke([{"role":"system","content":"You are a helpful assistant."},
                       {"role":"user","content": prompt}])
    return {"brief": resp}



In [18]:
# Graph Construction & Compile (fixed routing signature)

from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

# Assume you have State, check_missing, ask_clarification, receive_input, generate_brief defined

def route_from_check_missing(input_tuple):
    # Defensive unpacking for tuple/list/dict
    if isinstance(input_tuple, (tuple, list)):
        state = input_tuple[0]
        output = input_tuple[1]
    elif isinstance(input_tuple, dict):
        # If it's a dict, you may need to adjust keys based on what LangGraph passes
        state = input_tuple.get("state")
        output = input_tuple.get("output")
    else:
        raise TypeError(f"Unexpected input type: {type(input_tuple)}")
    if output and output.get("missing_field") is not None:
        return "ask_clarification"
    else:
        return "generate_brief"

builder = StateGraph(State)
builder.add_node("check_missing", check_missing)
builder.add_node("ask_clarification", ask_clarification)
builder.add_node("receive_input", receive_input)
builder.add_node("generate_brief", generate_brief)

builder.add_edge(START, "check_missing")

builder.add_conditional_edges(
    "check_missing",
    route_from_check_missing,
    {"ask_clarification": "ask_clarification", "generate_brief": "generate_brief"}
)

builder.add_edge("ask_clarification", "receive_input")
builder.add_edge("receive_input", "check_missing")
builder.add_edge("generate_brief", END)

graph = builder.compile(checkpointer=MemorySaver())


In [20]:
import uuid

thread_id = str(uuid.uuid4())

print("Agent ready. (type 'exit' or 'quit' to stop)")

# initialize state
state = {
    "user_request": None,
    "topic": None,
    "scope": None,
    "time_period": None,
    "preferred_sources": None,
    "format": None,
    "brief": None,
    "messages": []
}

while True:
    usr = input("You: ")
    if usr.lower() in {"exit","quit"}:
        print("Bye!")
        break
    if state["user_request"] is None:
        state["user_request"] = usr.strip()
    config = {"configurable": {"thread_id": thread_id}}

    result = graph.invoke(state, config=config)
    if isinstance(result, tuple):
        state = result[0]
    else:
        state = result

    print(type(state), state)

    if "question" in state:
        print("Agent:", state["question"])
        ans = input("You: ")
        state[state["missing_field"]] = ans.strip()
        continue

    if "brief" in state:
        brief = state["brief"].content if hasattr(state["brief"], "content") else state["brief"]
        print("=== Research Brief ===")
        print(brief)
        break


Agent ready. (type 'exit' or 'quit' to stop)
<class 'dict'> {'user_request': 'tell me about taj mahal', 'topic': None, 'scope': None, 'time_period': None, 'preferred_sources': None, 'format': None, 'brief': AIMessage(content="## Research Brief: The Taj Mahal\n\n### 1. Introduction and Overview\n\nThe Taj Mahal is an iconic ivory-white marble mausoleum located on the right bank of the Yamuna River in the Indian city of Agra. Commissioned in 1631 by the Mughal emperor Shah Jahan to house the tomb of his beloved wife, Mumtaz Mahal, it also enshrines the tomb of Shah Jahan himself. Recognized globally as a masterpiece of Mughal architecture and a symbol of India's rich history, it attracts millions of visitors annually and was designated a UNESCO World Heritage Site in 1983.\n\n### 2. Historical Context\n\n*   **Commissioning:** Emperor Shah Jahan, deeply saddened by the death of his wife Mumtaz Mahal during childbirth in 1631, began construction of the mausoleum in her memory.\n*   **Cons